# Explaining MinGPT 
> This blog explains MinGPT Model implementation on Addition Data, by <a href="https://github.com/karpathy/minGPT">Andrej Karpathy</a>.

- toc: true 
- badges: true
- comments: true
- sticky_rank: 1
- author: Rayan
- image: images/diagram.png
- categories: [MinGPT, transformers]

> `Objective : "To Train a GPT model on a dedicated addition dataset to see if a Transformer can learn to add."`

Our Objective is inspired by the addition section in the GPT-3 paper (Language Models are a few shot learners)- https://arxiv.org/pdf/2005.14165v4.pdf

Code References Used :
1. https://github.com/openai/gpt-2 - This has GPT code (only model code is taken but not training) in Tensorflow, which is converted to our usecase in PyTorch here.
2. https://github.com/openai/image-gpt 

## 1. Imports

In [1]:
# Imports
import math
import logging
import gc
import os
import numpy as np
import torchvision
import torch
import matplotlib.pyplot as plt
import random
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data.dataloader import DataLoader

## 2. Setting Our Seed :

In [3]:
# Seeding 
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Making deterministic, setting our seed
set_seed(42)

In order to generate data, we define our custom Addition Dataset Class. The sum of two n-digit numbers gives a third up to (n+1)-digit number. So our
encoding will simply be the n-digit first number, n-digit second number, 
and (n+1)-digit result, all simply concatenated together. Because each addition
problem is so structured, there is no need to bother the model with encoding
+, =, or other tokens. Each possible sequence has the same length, and simply
contains the raw digits of the addition problem.

As a few examples, the 2-digit problems:
- 85 + 50 = 135 becomes the sequence [8, 5, 5, 0, 1, 3, 5]
- 6 + 39 = 45 becomes the sequence [0, 6, 3, 9, 0, 4, 5]
etc.

We will also only train GPT on the final (n+1)-digits because the first
two n-digits are always assumed to be given. So when we give GPT an exam later,
we will e.g. feed it the sequence [0, 6, 3, 9], which encodes that we'd like
to add 6 + 39, and hope that the model completes the integer sequence with [0, 4, 5] in 3 sequential steps.

In [ ]:
from torch.utils.data import Dataset

class AdditionDataset(Dataset):
    """
    Our Custom Dataset Class for Generating Data into Training and Test sets.

    Returns addition problems of up to some number of digits in the inputs. We recall
    that all GPT cares about are sequences of integers, and completing them according to
    patterns in the data. Therefore, we have to somehow encode addition problems
    as a sequence of integers.
    
    """

    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1
        
        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __getitem__(self, idx):
        # given a problem index idx, first recover the associated a + b
        idx = self.ixes[idx]
        nd = 10**self.ndigit
        a = idx // nd
        b = idx %  nd
        c = a + b
        render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028" 
        dix = [int(s) for s in render] # convert each character to its token index
        # x will be input to GPT and y will be the associated expected outputs
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long) # predict the next token in the sequence
        y[:self.ndigit*2-1] = -100 # we will only train in the output locations. -100 will mask loss to zero
        return x, y
